In [1]:
from requests_html import HTMLSession
import requests
import numpy as np
import re

In [31]:
from dataclasses import dataclass, asdict

In [3]:
s = HTMLSession()

In [4]:
!ls

2020-06-29_station_debug.ipynb sitemap_pc_cond_station_1.xml
2020-07-04_tabelog.ipynb       sitemap_pc_rst_station_1.xml
2020-07-12_Model.ipynb


In [5]:
import xml.etree.ElementTree as ET
with open("sitemap_pc_rst_station_1.xml") as f:
    tree = ET.parse(f)

In [6]:
root = tree.getroot()

In [7]:
items = list(root)

In [8]:
urls = [list(item)[0].text for item in items]

In [9]:
urls = [url for url in urls if "tokyo" in url]

In [10]:
urls = list(set(urls))

In [11]:
len(urls)

649

In [12]:
@dataclass
class Station:
    tabelog_url: str
    name: str
    top10_avarage_score: float
    num_shop: int

In [13]:
urls[:10]

['https://tabelog.com/tokyo/A1323/A132302/R5378/rstLst/',
 'https://tabelog.com/tokyo/A1317/A131701/R10340/rstLst/',
 'https://tabelog.com/tokyo/A1317/A131716/R4846/rstLst/',
 'https://tabelog.com/tokyo/A1323/A132304/R4890/rstLst/',
 'https://tabelog.com/tokyo/A1315/A131502/R8958/rstLst/',
 'https://tabelog.com/tokyo/A1309/A130902/R10511/rstLst/',
 'https://tabelog.com/tokyo/A1327/A132701/R9227/rstLst/',
 'https://tabelog.com/tokyo/A1326/A132602/R6100/rstLst/',
 'https://tabelog.com/tokyo/A1330/A133002/R1246/rstLst/',
 'https://tabelog.com/tokyo/A1309/A130905/R2293/rstLst/']

In [14]:
params = dict(
    SrtT="rt", sort_mode=1, Srt="D"
)

In [15]:
from time import sleep

In [16]:
from tqdm import tqdm_notebook

In [ ]:
errors = []
stations = []

In [24]:
tabelog_urls = [station.tabelog_url for station in stations]

In [26]:
for url in tqdm_notebook(urls):
    if url in tabelog_urls:
        continue
    try:
        resp = s.get(url, params=params)
        name = resp.html.find(".list-sidebar__item-title", first=True).text
        ratings = resp.html.find(".list-rst__rating-val")
        top10_avarage_score = np.mean([float(rating.text) for rating in ratings])
        count = resp.html.find(".c-page-count", first=True).text
        num_shop = re.findall("\d+", count)[-1]
        station = Station(url, name, top10_avarage_score, num_shop)
        stations.append(station)
    except:
        print(url)
    sleep(1)

<ipython-input-26-8c0109bbf12a>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url in tqdm_notebook(urls):


In [27]:
%store stations

Stored 'stations' (list)


In [33]:
dict_stations = [asdict(s) for s in stations]

In [38]:
!pip install pip install python-Levenshtein

     |████████████████████████████████| 48 kB 24 kB/s eta 0:00:01
Using legacy setup.py install for python-Levenshtein, since package 'wheel' is not installed.
    Running setup.py install for python-Levenshtein ... done


In [39]:
import Levenshtein

ModuleNotFoundError: No module named 'Levenshtein'

In [ ]:
Levenshtein.distance('Rievenstein', 'Levenshtein')

In [35]:
import pandas as pd

In [37]:
pd.DataFrame(dict_stations).to_csv("tmp.csv", index=False)